In [ ]:
import os
import time
import random
from queue import Queue
from tqdm import tqdm
import threading
import openai

from src.data import load_dataset
from src.utils import *
from src.prompt import format_judge_answers

In [ ]:
from openai import OpenAI
client=OpenAI(api_key='<KEY>')
client.api_key='...'
client.base_url='...'


In [ ]:
config = read_json(os.path.join(".\configs", "rewardbench.json"))
ds = load_dataset(config["dataset_key"], cache_dir=config["instructions_responses_pairs_data_path"])
ds = [_ for _ in ds["filtered"]]
new_ds = []
for case in ds:
    if random.uniform(0, 1) < 0.5:
        case["response a"] = case["chosen"]
        case["response b"] = case["rejected"]
        case["choice_label"] = 1
    else:
        case["response a"] = case["rejected"]
        case["response b"] = case["chosen"]
        case["choice_label"] = 2
    new_ds.append(case)
ds = new_ds

In [ ]:

class Crawl_thread(threading.Thread):
    def __init__(self, thread_id, queue):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        #self.filename = target.replace("*", "_thread" + str(self.thread_id) + ".jsonl")
        self.queue = queue
    
    def run(self):
        print("start thread:", self.thread_id)
        self.crawl_spider()
        print("quit thread:", self.thread_id)
    
    def crawl_spider(self):
        global all_get_data3, candidate_key
        while True:
            if self.queue.empty():
                break
            else:
                row= self.queue.get()
                #msgs=row["prompt_new"]
                msgs = row #self.openai_template(row[0], row[1])
                
                try:
                    success = False
                    for attempt in range(5):
                        try:
                            #client.api_key = random.choice(all_keys)
                            response = client.chat.completions.create(
                                model = "gpt-4o",
                                messages=msgs, 
                                temperature=0.0
                            )
                        except Exception as e:
                            time.sleep(random.randint(1, 30))
                            print(f'{e}')
                        except openai.error.APIerror:
                            time.sleep(random.randint(1, 30))
                            print(f"API GG")
                        else:
                            success=True
                            break
                    if success:
                        res = response.choices[0].message.content
                        row.append(res)
                        
                except:
                    pass


# Reviseval

In [ ]:
revise_v = (
    "Please act as a powerful reviser to revise the response generated by an AI "
    "assistant to the instruction displayed below. You should revise the response to "
    "follow the user's instructions and answer the user's instruction better. Your "
    "revision should focus on accuracy and whether the response honestly/precisely/closely executes the instruction. If the original response "
    "is good enough, simply output the original response. \n"
    "[Instruction]\n{instruction}\n"
    "[The Start of Assistant's Answer]\n{response_output_1}\n[The End of Assistant's Answer]\n"
    "I also give you another model-generated response, which is not necessarily of"
    "better quality, as a reference for your revision, and you can draw on its"
    "strengths and avoid its weaknesses.\n"
    "[Another Response]\n{response_output_2}\n"
    #"Do NOT provide any explanation for your response."
    "ONLY output the complete revised response without saying anything else."
)



In [ ]:
def format_revise_answers(instruction, answer_a, answer_b, multi_turn=False, model_modifier=None):
    kwargs = {}
    
    user_prompt = revise_v.format(
        instruction=instruction,
        response_output_1=answer_a[1]["content"],
        response_output_2=answer_b[1]["content"],
        **kwargs,
    )
    
    # gemini adds what was the system prompt before the content, and has no system prompt


    return user_prompt


In [ ]:

thread_inputs = []
for case in tqdm(ds):

    user_prompt = format_revise_answers(case["prompt"], [{"content": case["prompt"]}, {"content": case["response b"]}], [{"content": case["prompt"]}, {"content": case["response a"]}], False, None)
    thread_inputs.append([{"role": "user", "content": user_prompt}])
class Crawl_thread(threading.Thread):
    def __init__(self, thread_id, queue):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        self.queue = queue
    
    def run(self):
        print("start thread:", self.thread_id)
        self.crawl_spider()
        print("quit thread:", self.thread_id)
    
    def crawl_spider(self):
        global all_get_data3, candidate_key
        while True:
            if self.queue.empty():
                break
            else:
                row= self.queue.get()
                #msgs=row["prompt_new"]
                msgs = row #self.openai_template(row[0], row[1])
                try:
                    success = False
                    for attempt in range(5):
                        try:
                            #client.api_key = random.choice(all_keys)
                            response = client.chat.completions.create(
                                model = "gpt-4o",
                                messages=msgs, 
                                temperature=0.3
                            )
                        except Exception as e:
                            time.sleep(random.randint(1, 30))
                            print(f'{e}')
                        except openai.error.APIerror:
                            time.sleep(random.randint(1, 30))
                            print(f"API GG")
                        else:
                            success=True
                            break
                    if success:
                        res = response.choices[0].message.content
                        row.append(res)
                        
                except:
                    pass

all_data = thread_inputs
pageQueue = Queue(len(all_data))
for p in all_data:
    pageQueue.put(p)
print(pageQueue.qsize())

crawl_threads = []
crawl_name_list = range(50)
for thread_id in crawl_name_list:
    thread = Crawl_thread(thread_id, pageQueue)
    time.sleep(0.5)
    thread.start()
    crawl_threads.append(thread)
for thread in crawl_threads:
    thread.join()  # Wait for each thread to finish

In [ ]:
new_content = []
for case, x in zip(thread_inputs, ds):
    x["reference"] = case[-1]
    print(case[-1])
    new_content.append(x)

In [ ]:
prompt_reference_v2 = (
    "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user's instructions and answers the user's "
    "question better. our evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. And I also give a reliable reference answer, and begin your "
    "evaluation by comparing the two responses with the reference answer and provide a short explanation. Avoid any position biases and ensure that the order in which "
    "the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names "
    "of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "
    '"[[A]]" if assistant A is better, "[[B]]" if assistant B is better.'
)


MTBENCH_REFERENCE_V2 = {
    "name": "pair-v2",
    "type": "pairwise",
    "system_prompt": prompt_reference_v2,
    "prompt_template": "[User Question]\n{question}\n\n[Reference Answer]\n{ref_answer}\n\n[The Start of Assistant A's Answer]\n{answer_a}\n[The End of Assistant A's Answer]\n\n[The Start of Assistant B's Answer]\n{answer_b}\n[The End of Assistant B's Answer]",
    "description": "Prompt for general questions",
    "category": "general",
    "output_format": "[[A]]",
}

In [ ]:
def format_judge_reference_answers(instruction, reference, answer_a, answer_b, multi_turn=False, model_modifier=None):
    kwargs = {}
    
    system_prompt = MTBENCH_REFERENCE_V2["system_prompt"]
    user_prompt = MTBENCH_REFERENCE_V2["prompt_template"].format(
        question=instruction,
        ref_answer=reference,
        answer_a=answer_a[1]["content"],
        answer_b=answer_b[1]["content"],
        **kwargs,
    )

    # gemini adds what was the system prompt before the content, and has no system prompt
    

    return system_prompt, user_prompt

thread_inputs = []
for case in tqdm(new_content):
    system_prompt, user_prompt = format_judge_reference_answers(case["prompt"], case["reference"], [{"content": case["prompt"]}, {"content": case["response a"]}], [{"content": case["prompt"]}, {"content": case["response b"]}], False, None)
    thread_inputs.append([{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}])

In [ ]:
class Crawl_thread(threading.Thread):
    def __init__(self, thread_id, queue):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        #self.filename = target.replace("*", "_thread" + str(self.thread_id) + ".jsonl")
        self.queue = queue
    
    def run(self):
        print("start thread:", self.thread_id)
        self.crawl_spider()
        print("quit thread:", self.thread_id)
    
    def crawl_spider(self):
        global all_get_data3, candidate_key
        while True:
            if self.queue.empty():
                break
            else:
                row= self.queue.get()
                #msgs=row["prompt_new"]
                msgs = row #self.openai_template(row[0], row[1])
                
                try:
                    success = False
                    for attempt in range(5):
                        try:
                            #client.api_key = random.choice(all_keys)
                            response = client.chat.completions.create(
                                model = "gpt-4o",
                                messages=msgs, 
                                temperature=0.1
                            )
                        except Exception as e:
                            time.sleep(random.randint(1, 30))
                            print(f'{e}')
                        except openai.error.APIerror:
                            time.sleep(random.randint(1, 30))
                            print(f"API GG")
                        else:
                            success=True
                            break
                    if success:
                        res = response.choices[0].message.content
                        row.append(res)
                        
                except:
                    pass

all_data = thread_inputs
pageQueue = Queue(len(all_data))
for p in all_data:
    pageQueue.put(p)
print(pageQueue.qsize())

crawl_threads = []
crawl_name_list = range(50)
for thread_id in crawl_name_list:
    thread = Crawl_thread(thread_id, pageQueue)
    time.sleep(0.5)
    thread.start()
    crawl_threads.append(thread)
for thread in crawl_threads:
    thread.join() 

In [ ]:
AA, AB, BA, BB = 0, 0, 0, 0
for x, case in zip(thread_inputs, new_content):
    #判断Flip
    if x[-1] is None:
        COUNT+=1
        continue
    case["round2_judgment"] = x[-1]

    
    if "[[A]]" in case["round2_judgment"] and case["choice_label"] == 1:
        AA+=1
    elif "[[B]]" in case["round2_judgment"] and case["choice_label"] == 2:
        BB += 1
    elif "[[B]]" in case["round2_judgment"] and case["choice_label"] == 1:
        AB +=1
    elif "[[A]]" in case["round2_judgment"] and case["choice_label"] == 2:
        BA+=1
print(AA,BB,AB,BA)


In [ ]:
PARTITION=[358, 358+456, 358+456+740, 1431]
VANILLA_SUBSETS = [0, 0, 0, 0]
NEW_SUBSETS = [0, 0, 0, 0]
def parttt(nid):
    if nid < PARTITION[0]:
        return 0
    elif nid < PARTITION[1] and nid > PARTITION[0]:
        return 1
    elif nid < PARTITION[2] and nid > PARTITION[1]:
        return 2
    else:
        return 3
COUNT=0


for x, case in zip(thread_inputs, new_content):
    #判断Flip
    if x[-1] is None:
        COUNT+=1
        continue
    case["round2_judgment"] = x[-1]

    subset = parttt(COUNT)
    
    if "[[A]]" in case["round2_judgment"] and case["choice_label"] == 1:
        NEW_SUBSETS[subset]+=1
    COUNT+=1

print("NEW:", NEW_SUBSETS[0]/358, NEW_SUBSETS[1]/456, NEW_SUBSETS[2]/740, NEW_SUBSETS[3]/1431,sum(NEW_SUBSETS)/len(new_content))